In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cyber-test/test (1).csv
/kaggle/input/cyber-train/train (1).csv


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import pandas as pd

# Load dataset
file_path = '/kaggle/input/cyber-train/train (1).csv'
data = pd.read_csv(file_path)

# Preprocess the dataset
data = data.dropna(subset=['category', 'crimeaditionalinfo'])
X = data['crimeaditionalinfo']
y = data['category']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Preprocess data for Hugging Face
def preprocess_data(texts, labels, max_length=128):
    encodings = tokenizer(list(texts), truncation=True, padding=True, max_length=max_length)
    inputs = torch.tensor(encodings['input_ids'])
    masks = torch.tensor(encodings['attention_mask'])
    labels = torch.tensor(labels)
    return inputs, masks, labels

train_inputs, train_masks, train_labels = preprocess_data(X_train, y_train)
test_inputs, test_masks, test_labels = preprocess_data(X_test, y_test)

# Define a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, masks, labels):
        self.inputs = inputs
        self.masks = masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx],
            'attention_mask': self.masks[idx],
            'labels': self.labels[idx]
        }

train_dataset = CustomDataset(train_inputs, train_masks, train_labels)
test_dataset = CustomDataset(test_inputs, test_masks, test_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50
)

# Define Trainer
from transformers import Trainer, EvalPrediction
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred: EvalPrediction):
    preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

# Example Inference
text_example = ["Describe a recent incident of cyber fraud."]
inputs = tokenizer(text_example, truncation=True, padding=True, max_length=128, return_tensors="pt")
outputs = model(**inputs)
predicted_class = torch.argmax(outputs.logits, dim=1).item()
predicted_label = label_encoder.inverse_transform([predicted_class])[0]
print("Predicted Category:", predicted_label)


In [ ]:
import torch

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move the model to the appropriate device

# Example Inference
text_example = ["bruh"]
inputs = tokenizer(text_example, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Move inputs to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Perform inference
with torch.no_grad():  # Disable gradient computation for inference
    outputs = model(**inputs)

# Get the predicted class
predicted_class = torch.argmax(outputs.logits, dim=1).item()
predicted_label = label_encoder.inverse_transform([predicted_class])[0]
print("Predicted Category:", predicted_label)


In [ ]:
pip install torch_xla transformers datasets


In [ ]:
pip install imbalanced-learn


In [ ]:
pip install tiktoken sentencepiece


In [ ]:
import torch
import torch_xla.core.xla_model as xm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Load dataset
file_path = '/kaggle/input/cyber-train/train (1).csv'
data = pd.read_csv(file_path)

# Preprocess dataset
data = data.dropna(subset=['category', 'crimeaditionalinfo'])
X = data['crimeaditionalinfo']
y = data['category']

# Handle rare classes
class_counts = y.value_counts()
rare_classes = class_counts[class_counts < 2].index
data = data[~data['category'].isin(rare_classes)]
X = data['crimeaditionalinfo']
y = data['category']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Oversample to balance classes
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(pd.DataFrame(X), y_encoded)

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled.squeeze(), y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)

# Replace DeBERTa-v3 with RoBERTa-base
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=len(label_encoder.classes_)
)

# Move model to TPU
device = xm.xla_device()
model = model.to(device)

# Preprocess data
def preprocess_data(texts, labels, max_length=512):
    encodings = tokenizer(list(texts), truncation=True, padding=True, max_length=max_length)
    inputs = torch.tensor(encodings['input_ids'])
    masks = torch.tensor(encodings['attention_mask'])
    labels = torch.tensor(labels)
    return inputs, masks, labels

train_inputs, train_masks, train_labels = preprocess_data(X_train, y_train)
test_inputs, test_masks, test_labels = preprocess_data(X_test, y_test)

# Custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, masks, labels):
        self.inputs = inputs
        self.masks = masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx],
            'attention_mask': self.masks[idx],
            'labels': self.labels[idx]
        }

train_dataset = CustomDataset(train_inputs, train_masks, train_labels)
test_dataset = CustomDataset(test_inputs, test_masks, test_labels)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    per_device_train_batch_size=16,  # TPU-compatible batch size
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    warmup_steps=500,
    report_to="none",
    tpu_num_cores=8  # TPU utilization
)

# Metrics for evaluation
def compute_metrics(eval_pred):
    preds = eval_pred.predictions.argmax(axis=1)
    labels = eval_pred.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)


In [3]:
pip install sentence-transformers


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 2.5 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import torch
import numpy as np
from collections import Counter

# Load dataset
file_path = '/kaggle/input/cyber-train/train (1).csv'  # Update this path if needed
data = pd.read_csv(file_path)

# Preprocess the dataset
data = data.dropna(subset=['category', 'crimeaditionalinfo'])
data['cleaned_text'] = data['crimeaditionalinfo'].str.lower().str.replace(r'\W', ' ')

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['category'])

# Check class distribution
class_counts = Counter(y)
print("Initial class distribution:")
for label, count in sorted(class_counts.items()):
    print(f"Class {label}: {count}")

# Remove classes with only one sample
min_samples = 5
valid_classes = [class_ for class_, count in class_counts.items() if count > min_samples]
mask = np.isin(y, valid_classes)
X_filtered = data['cleaned_text'][mask].reset_index(drop=True)
y_filtered = y[mask]

# Re-encode labels to ensure consecutive integers
label_encoder_filtered = LabelEncoder()
y_filtered = label_encoder_filtered.fit_transform(y_filtered)

print(f"\nNumber of classes after filtering: {len(label_encoder_filtered.classes_)}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_filtered, test_size=0.2, random_state=42, stratify=y_filtered
)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create torch dataset
class CyberDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(
            texts.tolist(),
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors="pt"
        )
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = CyberDataset(X_train, y_train, tokenizer)
test_dataset = CyberDataset(X_test, y_test, tokenizer)

# Initialize model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_encoder_filtered.classes_)
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2
)

# Define metrics computation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
print("\nStarting training...")
trainer.train()

# Evaluate the model
print("\nEvaluating model...")
eval_results = trainer.evaluate()
print("\nEvaluation Results:", eval_results)

# Save the model and label encoder
model.save_pretrained('./cyber_classifier_model')
tokenizer.save_pretrained('./cyber_classifier_tokenizer')
import joblib
joblib.dump(label_encoder_filtered, './label_encoder.joblib')

Initial class distribution:
Class 0: 10877
Class 1: 379
Class 2: 480
Class 3: 3608
Class 4: 161
Class 5: 1710
Class 6: 183
Class 7: 57416
Class 8: 444
Class 9: 12138
Class 10: 56
Class 11: 2822
Class 12: 1
Class 13: 1552
Class 14: 1838

Number of classes after filtering: 14


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(



Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.680000,0.665983,0.765120,0.743041,0.730527,0.765120
2,0.582200,0.659882,0.776491,0.753294,0.753103,0.776491
3,0.581100,0.699082,0.773715,0.756031,0.748054,0.773715


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Evaluating model...


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Evaluation Results: {'eval_loss': 0.6990817189216614, 'eval_accuracy': 0.7737148347835371, 'eval_f1': 0.7560313696885839, 'eval_precision': 0.748053619165972, 'eval_recall': 0.7737148347835371, 'eval_runtime': 63.8586, 'eval_samples_per_second': 293.351, 'eval_steps_per_second': 18.337, 'epoch': 3.0}


['./label_encoder.joblib']

In [8]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import joblib

def load_model_and_tokenizer(model_path='./cyber_classifier_model', 
                           tokenizer_path='./cyber_classifier_tokenizer',
                           label_encoder_path='./label_encoder.joblib'):
    """
    Load the saved model, tokenizer and label encoder
    """
    # Load tokenizer and model
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    model = BertForSequenceClassification.from_pretrained(model_path)
    label_encoder = joblib.load(label_encoder_path)
    
    # Set model to evaluation mode
    model.eval()
    
    return model, tokenizer, label_encoder

def predict_category(text, model, tokenizer, label_encoder):
    """
    Predict category for a given text
    """
    # Ensure text is in list format
    if isinstance(text, str):
        text = [text]
    
    # Tokenize input
    inputs = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )
    
    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get predicted class
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    # Convert to label
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    # Get confidence scores
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    confidence = probabilities[0][predicted_class].item()
    
    return {
        'predicted_label': predicted_label,
        'confidence': confidence,
        'class_probabilities': probabilities[0].tolist()
    }

# Example usage
if __name__ == "__main__":
    # Load model, tokenizer, and label encoder
    model, tokenizer, label_encoder = load_model_and_tokenizer()
    
    # Example texts
    example_texts = [
        "Unauthorized access detected to company database through SQL injection",
        "Received a suspicious email asking for bank details",
        "DDoS attack targeting our web servers"
    ]
    
    # Make predictions
    print("Making predictions...")
    for text in example_texts:
        result = predict_category(text, model, tokenizer, label_encoder)
        print("\nText:", text)
        print("Predicted Category:", result['predicted_label'])
        print(f"Confidence: {result['confidence']*100:.2f}%")

    # Single prediction example
    single_text = "Describe a recent incident of cyber fraud."
    result = predict_category(single_text, model, tokenizer, label_encoder)
    print("\nSingle Prediction Example:")
    print("Text:", single_text)
    print("Predicted Category:", result['predicted_label'])
    print(f"Confidence: {result['confidence']*100:.2f}%")

Making predictions...

Text: Unauthorized access detected to company database through SQL injection
Predicted Category: 7
Confidence: 95.10%

Text: Received a suspicious email asking for bank details
Predicted Category: 7
Confidence: 99.22%

Text: DDoS attack targeting our web servers
Predicted Category: 5
Confidence: 59.55%

Single Prediction Example:
Text: Describe a recent incident of cyber fraud.
Predicted Category: 7
Confidence: 97.77%
